In [3]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("~/.config/gcloud/application_default_credentials.json")

client = bigquery.Client(
    project="product-data-sandbox",
    default_query_job_config=bigquery.QueryJobConfig(
        default_dataset="product-data-sandbox.dataset1" 
    )
)

In [4]:
query = """
SELECT * FROM users ORDER BY created_at DESC LIMIT 10
 """

query_job = client.query(query)
result = query_job.to_dataframe()
result

,user_id,created_at,gender,age,city,country,signup_source,last_active
0,user010,2023-10-05 12:25:00+00:00,MALE,38,TOKYO,JAPAN,WEB,2023-11-21 16:15:00+00:00
1,user009,2023-09-30 10:40:00+00:00,FEMALE,41,SAPPORO,JAPAN,MOBILE_APP,2023-11-19 11:30:00+00:00
2,user008,2023-08-22 15:55:00+00:00,MALE,33,NAGOYA,JAPAN,WEB,2023-11-20 14:20:00+00:00
3,user007,2023-07-17 13:10:00+00:00,FEMALE,27,TOKYO,JAPAN,ADVERTISEMENT,2023-11-16 15:40:00+00:00
4,user006,2023-06-08 08:30:00+00:00,MALE,50,TOKYO,JAPAN,WEB,2023-11-17 10:05:00+00:00
5,user005,2023-05-12 11:20:00+00:00,FEMALE,35,KYOTO,JAPAN,MOBILE_APP,2023-11-18 17:10:00+00:00
6,user004,2023-04-25 16:45:00+00:00,MALE,29,TOKYO,JAPAN,REFERRAL,2023-11-21 12:45:00+00:00
7,user003,2023-03-10 09:15:00+00:00,MALE,42,OSAKA,JAPAN,WEB,2023-11-19 18:30:00+00:00
8,user002,2023-02-15 14:30:00+00:00,FEMALE,31,TOKYO,JAPAN,MOBILE_APP,2023-11-20 09:12:00+00:00
9,user001,2023-01-01 10:00:00+00:00,MALE,25,TOKYO,JAPAN,WEB,2023-11-15 14:25:00+00:00


In [5]:
query = """
SELECT * FROM events ORDER BY created_at DESC LIMIT 10
 """

query_job = client.query(query)
result = query_job.to_dataframe()
result

,event_id,created_at,user_id,event_name
0,None,2025-03-07 14:53:31+00:00,user003,LOGIN
1,None,2025-03-07 14:53:31+00:00,user001,LOGIN
2,None,2025-03-07 14:53:31+00:00,user006,LOGIN
3,None,2025-03-07 14:53:31+00:00,user010,LOGIN
4,None,2025-03-07 14:53:31+00:00,user008,LOGIN
5,None,2025-03-07 14:53:31+00:00,user004,LOGIN
6,None,2025-03-07 11:53:31+00:00,user004,BEGIN_CHECKOUT
7,None,2025-03-07 02:53:31+00:00,user010,LOGIN
8,None,2025-03-06 14:53:31+00:00,user006,CONFIRM
9,None,2025-03-06 14:53:31+00:00,user001,LOGIN


In [20]:
# Create test data cell
from datetime import datetime, timedelta

# Calculate current time and time 7 days ago (for recent events)
now = datetime.now()
seven_days_ago = now - timedelta(days=7)

# Create user test data
users_query = """
INSERT INTO dataset1.users (user_id, created_at, gender, age, city, country, signup_source, last_active)
VALUES
  ('user001', TIMESTAMP('2023-01-01 10:00:00'), 'MALE', 25, 'TOKYO', 'JAPAN', 'WEB', TIMESTAMP('2023-11-15 14:25:00')),
  ('user002', TIMESTAMP('2023-02-15 14:30:00'), 'FEMALE', 31, 'TOKYO', 'JAPAN', 'MOBILE_APP', TIMESTAMP('2023-11-20 09:12:00')),
  ('user003', TIMESTAMP('2023-03-10 09:15:00'), 'MALE', 42, 'OSAKA', 'JAPAN', 'WEB', TIMESTAMP('2023-11-19 18:30:00')),
  ('user004', TIMESTAMP('2023-04-25 16:45:00'), 'MALE', 29, 'TOKYO', 'JAPAN', 'REFERRAL', TIMESTAMP('2023-11-21 12:45:00')),
  ('user005', TIMESTAMP('2023-05-12 11:20:00'), 'FEMALE', 35, 'KYOTO', 'JAPAN', 'MOBILE_APP', TIMESTAMP('2023-11-18 17:10:00')),
  ('user006', TIMESTAMP('2023-06-08 08:30:00'), 'MALE', 50, 'TOKYO', 'JAPAN', 'WEB', TIMESTAMP('2023-11-17 10:05:00')),
  ('user007', TIMESTAMP('2023-07-17 13:10:00'), 'FEMALE', 27, 'TOKYO', 'JAPAN', 'ADVERTISEMENT', TIMESTAMP('2023-11-16 15:40:00')),
  ('user008', TIMESTAMP('2023-08-22 15:55:00'), 'MALE', 33, 'NAGOYA', 'JAPAN', 'WEB', TIMESTAMP('2023-11-20 14:20:00')),
  ('user009', TIMESTAMP('2023-09-30 10:40:00'), 'FEMALE', 41, 'SAPPORO', 'JAPAN', 'MOBILE_APP', TIMESTAMP('2023-11-19 11:30:00')),
  ('user010', TIMESTAMP('2023-10-05 12:25:00'), 'MALE', 38, 'TOKYO', 'JAPAN', 'WEB', TIMESTAMP('2023-11-21 16:15:00'))
"""

# Create event test data
events_query = f"""
INSERT INTO dataset1.events (user_id, event_name, created_at)
VALUES
  -- user001 events (meets criteria: male, Tokyo, 5+ LOGIN)
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'LOGIN', TIMESTAMP('{(now - timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'PURCHASE', TIMESTAMP('{(now - timedelta(days=2, hours=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'VIEW_PDP', TIMESTAMP('{(now - timedelta(days=2, hours=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user001', 'ADD_TO_CART', TIMESTAMP('{(now - timedelta(days=3, hours=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  -- user004 events (meets criteria: male, Tokyo, 5 LOGIN)
  ('user004', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'LOGIN', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'LOGIN', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'LOGIN', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'LOGIN', TIMESTAMP('{(now - timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'PURCHASE', TIMESTAMP('{(now - timedelta(days=3, hours=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'VIEW_PDP', TIMESTAMP('{(now - timedelta(days=2, hours=5)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user004', 'BEGIN_CHECKOUT', TIMESTAMP('{(now - timedelta(days=1, hours=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  -- user006 events (doesn't meet criteria: only 4 LOGIN)
  ('user006', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user006', 'LOGIN', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user006', 'LOGIN', TIMESTAMP('{(now - timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user006', 'LOGIN', TIMESTAMP('{(now - timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user006', 'CONFIRM', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user006', 'VIEW_PDP', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  -- user010 events (meets criteria: male, Tokyo, 5+ LOGIN)
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1, hours=12)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=5)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'LOGIN', TIMESTAMP('{(now - timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'CONFIRM', TIMESTAMP('{(now - timedelta(days=2, hours=6)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'PURCHASE', TIMESTAMP('{(now - timedelta(days=3, hours=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user010', 'BEGIN_CHECKOUT', TIMESTAMP('{(now - timedelta(days=4, hours=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  -- Other users' diverse events
  ('user002', 'LOGIN', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user002', 'PURCHASE', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user002', 'VIEW_PDP', TIMESTAMP('{(now - timedelta(days=2, hours=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  ('user003', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user003', 'LOGIN', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user003', 'VIEW_PDP', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  ('user005', 'LOGIN', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user005', 'ADD_TO_CART', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  ('user007', 'LOGIN', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user007', 'BEGIN_CHECKOUT', TIMESTAMP('{(now - timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  ('user008', 'LOGIN', TIMESTAMP('{(now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user008', 'CONFIRM', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  
  ('user009', 'PURCHASE', TIMESTAMP('{(now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M:%S")}')),
  ('user009', 'ADD_TO_CART', TIMESTAMP('{(now - timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S")}'))
"""

# Execute query to insert user data
try:
    users_job = client.query(users_query)
    users_job.result()  # Wait for query to complete
    print("User data inserted successfully!")
except Exception as e:
    print(f"Error inserting user data: {e}")

# Execute query to insert event data
try:
    events_job = client.query(events_query)
    events_job.result()  # Wait for query to complete
    print("Event data inserted successfully!")
except Exception as e:
    print(f"Error inserting event data: {e}")

# Verify data has been inserted
verification_query = """
SELECT 'users' as table_name, COUNT(*) as count FROM dataset1.users
UNION ALL
SELECT 'events' as table_name, COUNT(*) as count FROM dataset1.events
"""

verification_job = client.query(verification_query)
verification_job.to_dataframe()

User data inserted successfully!
Event data inserted successfully!


,table_name,count
0,users,10
1,events,47


In [7]:
# Query using question mark placeholders
query = """
(SELECT user_id FROM users WHERE gender = ?)
INTERSECT DISTINCT
(SELECT user_id FROM users WHERE city LIKE CONCAT(?, '%'))
INTERSECT DISTINCT
(SELECT user_id 
 FROM events 
 WHERE event_name = ? 
   AND created_at >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
 GROUP BY user_id 
 HAVING COUNT(1) >= ?)
 """

# Execute query
query_job = client.query(query, job_config=bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter(None, "STRING", "MALE"),
        bigquery.ScalarQueryParameter(None, "STRING", "TOKYO"),
        bigquery.ScalarQueryParameter(None, "STRING", "LOGIN"), 
        bigquery.ScalarQueryParameter(None, "INT64", "5"),
    ]
))

result = query_job.to_dataframe()
result

,user_id
0,user004
1,user010
2,user001
